In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/temp.csv')
df

In [ ]:
data = df.iloc[:,1:]
target = df.iloc[:,:1]
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=34)
print(len(x_train), len(x_valid))
print(len(y_train), len(y_valid))

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

param = {
    'learning_rate': [0.1], #0.05 0.07 0.1 test 0.1 best
    'max_depth': [7], # 5,6,7 test 7 best
    'n_estimators': [650], #550, 600, 650 ,700 test 650 best
    'colsample_bytree':[1], #0.5 0.7 1 test 1 best
    'min_child_weight':[3], #1 2 3 test 3 best
    'subsample':[0.8] # 0.6 0.7 0.8 test 0.8 best
}
model = xgb.XGBRegressor(tree_method='gpu_hist')
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=5, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(x_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = grid_search.predict(x_train)
pred_val = grid_search.predict(x_valid)

print('train mae score: ', mean_absolute_error(y_train, pred_train))
print('val mae score:', mean_absolute_error(y_valid, pred_val))

In [ ]:
plt.figure(figsize=(300,100))
plt.plot(range(0, len(y_valid)), y_valid,'o-', label='Validation Actual')
plt.plot(range(0, len(pred_val)), pred_val, '-', label='Validation Predict')
plt.title('Prediction of House Prices')
plt.ylabel('Prices')
plt.legend()

In [ ]:
import pickle

# 학습된 모델 저장
with open('my_model.pkl', 'wb') as f:
    pickle.dump(grid_search.best_estimator_, f)

In [ ]:
# 저장된 모델 불러오기
with open('my_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [ ]:
loaded_model

In [ ]:
loaded_model.predict(x_valid)

In [ ]:
x_valid.shape

In [ ]:
x_valid

In [ ]:
y_valid